# X-ray photoelectron spectroscopy (XPS)

## Visualizing the WorkGraph Builder


In [1]:
from workgraph_collections.ase.espresso.xps import xps_workgraph
from aiida import load_profile
load_profile()

node = xps_workgraph.node()
node.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.ase.espresso.xps import xps_workgraph
from aiida import load_profile
load_profile()

wg = xps_workgraph()
wg.to_html()

## Example: ETFA molecule

### Prepare the inputs and submit the workflow


In [3]:
from ase.io import read
from aiida import load_profile
from workgraph_collections.ase.espresso.xps import xps_workgraph

load_profile()

# create input structure node
atoms = read("../datas/ETFA.xyz")
atoms.center(vacuum=3.0)
input_data = {
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 30,
            "ecutrho": 240,
            "occupations": "fixed",
        },
    }
kpts = (1, 1, 1)
# Pseudos from https://github.com/superstar54/xps-data/tree/main/pseudo_demo/pseudo_demo_pbe
pseudopotentials = {
    "C": "C.pbe-n-kjgipaw_psl.1.0.0.UPF",
    "H": "H.pbe-kjpaw_psl.1.0.0.UPF",
    "O": "O.pbe-n-kjpaw_psl.0.1.UPF",
    "F": "F.pbe-n-rrkjus_psl.1.0.0.UPF",
}
core_hole_pseudos = {
    "C_1s": "C.star1s.pbe-n-kjgipaw_psl.1.0.0.UPF",
    "O_1s": "O.star1s.pbe-n-kjpaw_psl.0.1.UPF",
}
#
metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
scf_inputs = {
    "command": "mpirun -np 4 pw.x",
    "computer": "localhost",
    "input_data": input_data,
    "kpts": kpts,
    "pseudopotentials": pseudopotentials,
    "pseudo_dir": "/home/wang_x3/datas/pseudos/xps/pbe",
    "core_hole_pseudos": core_hole_pseudos,
    "is_molecule": True,
}
# corrections from https://github.com/superstar54/xps-data/blob/main/pseudo_demo/pseudo_demo_pbe/datas.py
corrections = {
        "C": 345.99 - 6.2,
        "O": 676.47 - 8.25,
        }
# ===============================================================================
wg = xps_workgraph(
    atoms=atoms,
    atoms_list=[(0, "1s"), (1, "1s"), (2, "1s"), (3, "1s")],
    scf_inputs=scf_inputs,
    corrections=corrections,
    metadata=metadata,
)
wg.name = "ASE-espresso-XPS-ETFA"
wg.submit(wait=True, timeout=300)

WorkGraph node created, PK: 17649


<WorkChainNode: uuid: ab76064a-9ae0-4292-931f-3a3639e89998 (pk: 17649) (aiida_workgraph.engine.workgraph.WorkGraphEngine)>

## Print the results


In [4]:
for key, energy in wg.nodes["binding_energy"].outputs["result"].value.value.items():
    print(f"{key}: {energy:.2f} eV")

C_1s_0: 298.96 eV
C_1s_1: 295.86 eV
C_1s_2: 293.56 eV
C_1s_3: 291.85 eV
